In [1]:
!unzip -q /content/drive/MyDrive/CalculusSegmenation.zip -d CalculusSegmenation



In [2]:
%cd /content/CalculusSegmenation/CalculusSegmenation

/content/CalculusSegmenation/CalculusSegmenation


In [11]:
# !pip install -U pip
# !pip install -U tensorflow==2.18.0 tf-keras segmentation-models --quiet

# !pip install segmentation-models-pytorch --quiet
# !pip install albumentations --quiet

#!pip install -U segmentation-models-pytorch albumentations opencv-python

!pip install segmentation-models-pytorch albumentations opencv-python tqdm --quiet


In [26]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
from pathlib import Path
from tqdm import tqdm  # <--- Added

torch.manual_seed(42)
np.random.seed(42)

# CONFIG
IMG_SIZE = (256, 256)
BATCH_SIZE = 4
EPOCHS = 30
best_val_iou = 0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATASET_ROOT = Path("output/train")  # change to output/test or output/valid when needed

# ==== DATASET ====
class ToothDataset(Dataset):
    def __init__(self, image_mask_pairs):
        self.pairs = image_mask_pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, mask_path = self.pairs[idx]
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, IMG_SIZE) / 255.0
        mask = cv2.resize(mask, IMG_SIZE) / 255.0
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
        return img, mask

# ==== LOAD IMAGE-MASK PAIRS ====
def get_pairs(dataset_root):
    pairs = []
    for folder in dataset_root.iterdir():
        data_dir = folder / "data"
        mask_dir = folder / "calculus_masks"
        if not data_dir.exists() or not mask_dir.exists():
            continue
        for img_file in data_dir.glob("tooth_*.png"):
            num = ''.join(filter(str.isdigit, img_file.stem))
            candidates = [
                mask_dir / f"calculus_mask_{int(num)}.png",
                mask_dir / f"calculus_mask_{int(num):02d}.png"
            ]
            for c in candidates:
                if c.exists():
                    pairs.append((str(img_file), str(c)))
                    break
    return pairs

# ==== METRIC ====
def compute_iou(preds, masks, threshold=0.5):
    preds = (preds > threshold).float()
    intersection = (preds * masks).sum((1, 2, 3))
    union = ((preds + masks) >= 1).float().sum((1, 2, 3))
    return ((intersection + 1e-6) / (union + 1e-6)).mean().item()

# ==== MAIN ====
pairs = get_pairs(DATASET_ROOT)
train_pairs, val_pairs = train_test_split(pairs, test_size=0.2, random_state=42)

train_loader = DataLoader(ToothDataset(train_pairs), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ToothDataset(val_pairs), batch_size=BATCH_SIZE)

model = smp.Unet(encoder_name="resnet50", in_channels=3, classes=1).to(DEVICE)
loss_fn = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Training]", ncols=100)
    for imgs, masks in train_loop:
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_loop.set_postfix(loss=f"{train_loss:.2f}")

    model.eval()
    val_loss, val_iou = 0, 0
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Validating]", ncols=100)
    with torch.no_grad():
        for imgs, masks in val_loop:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            preds = model(imgs)
            val_loss += loss_fn(preds, masks).item()
            val_iou += compute_iou(preds, masks)
            val_loop.set_postfix(iou=f"{val_iou / len(val_loader):.4f}")

    avg_val_iou = val_iou / len(val_loader)
    if avg_val_iou > best_val_iou:
        best_val_iou = avg_val_iou
        torch.save(model.state_dict(), "best_model.pth")

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val IoU: {avg_val_iou:.4f}")

torch.save(model.state_dict(), "best_model.pth")


Epoch 1/30 [Validating]: 100%|████████████████████████| 167/167 [00:05<00:00, 29.14it/s, iou=0.1141]


Epoch 1/30 - Train Loss: 354.9827, Val Loss: 92.0217, Val IoU: 0.1141


Epoch 2/30 [Validating]: 100%|████████████████████████| 167/167 [00:05<00:00, 29.83it/s, iou=0.4470]


Epoch 2/30 - Train Loss: 330.2125, Val Loss: 87.7509, Val IoU: 0.4470


Epoch 3/30 [Validating]: 100%|████████████████████████| 167/167 [00:05<00:00, 29.48it/s, iou=0.4596]


...

Epoch 28/30 - Train Loss: 264.5066, Val Loss: 65.3185, Val IoU: 0.5716


Epoch 29/30 [Validating]: 100%|███████████████████████| 167/167 [00:05<00:00, 29.01it/s, iou=0.4165]


Epoch 29/30 - Train Loss: 264.8786, Val Loss: 65.8178, Val IoU: 0.4165


Epoch 30/30 [Validating]: 100%|███████████████████████| 167/167 [00:05<00:00, 28.76it/s, iou=0.6826]


Epoch 30/30 - Train Loss: 261.4825, Val Loss: 63.7054, Val IoU: 0.6826


In [27]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from tqdm.notebook import tqdm
import segmentation_models_pytorch as smp

# CONFIG
IMG_SIZE = (256, 256)
BATCH_SIZE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEST_ROOT = Path("output/test")

# Dataset
class ToothDataset(torch.utils.data.Dataset):
    def __init__(self, image_mask_pairs):
        self.pairs = image_mask_pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, mask_path = self.pairs[idx]
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, IMG_SIZE) / 255.0
        mask = cv2.resize(mask, IMG_SIZE) / 255.0
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
        return img, mask

# Load image-mask pairs from nested folders
def get_pairs(dataset_root):
    pairs = []
    for folder in dataset_root.iterdir():
        data_dir = folder / "data"
        mask_dir = folder / "calculus_masks"
        if not data_dir.exists() or not mask_dir.exists():
            continue
        for img_file in data_dir.glob("tooth_*.png"):
            num = ''.join(filter(str.isdigit, img_file.stem))
            for mask_name in [f"calculus_mask_{int(num)}.png", f"calculus_mask_{int(num):02d}.png"]:
                mask_path = mask_dir / mask_name
                if mask_path.exists():
                    pairs.append((str(img_file), str(mask_path)))
                    break
    return pairs

# IoU metric
def compute_iou(preds, masks, threshold=0.5):
    preds = (preds > threshold).float()
    intersection = (preds * masks).sum((1, 2, 3))
    union = ((preds + masks) >= 1).float().sum((1, 2, 3))
    return ((intersection + 1e-6) / (union + 1e-6)).mean().item()

# Load test data
test_pairs = get_pairs(TEST_ROOT)
test_loader = DataLoader(ToothDataset(test_pairs), batch_size=BATCH_SIZE)

# Load model
model = smp.Unet(encoder_name="resnet50", in_channels=3, classes=1)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Evaluate
iou_total = 0
with torch.no_grad():
    for imgs, masks in tqdm(test_loader, desc="Evaluating"):
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
        preds = model(imgs)
        iou_total += compute_iou(preds, masks)

avg_iou = iou_total / len(test_loader)
print(f"Mean IoU on test set: {avg_iou:.4f}")


Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

Mean IoU on test set: 0.6147
